In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import catboost as cb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, ShuffleSplit,StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold,train_test_split,GroupShuffleSplit,StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score,mean_squared_error,mean_absolute_error,log_loss,confusion_matrix
import sqlite3
import xgboost as xgb
import datetime
from sklearn.linear_model import LogisticRegression
from scipy.stats import pearsonr
import gc
from sklearn.model_selection import TimeSeriesSplit
#from bayes_opt import BayesianOptimization
import re
from string import punctuation
from scipy.spatial import Voronoi
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
from tqdm.notebook import tqdm
from numba import jit
from collections import Counter
import json
import joblib
import multiprocessing
import time

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int32', 'int64', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
sample_submission = pd.read_csv('../data/sample_submission.csv')
sell_prices = pd.read_csv('../data/sell_prices.csv')
sales_train = pd.read_csv('../data/sales_train_validation.csv')
calendar = pd.read_csv('../data/calendar.csv')

In [4]:
####扩展sales_train df方便后续
#sales_train = pd.read_csv('../data/sales_train_validation.csv')
for _ in [f'd_{i}' for i in range(1914,1970)]:
    sales_train[_] = np.nan

In [5]:
sales_train_long_format = pd.melt(sales_train,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name = 'day_num',value_name='sale')

In [6]:
def transform_day_to_num(str1):
    return int(str1[2:])
sales_train_long_format['day_num'] = sales_train_long_format['day_num'].map(transform_day_to_num)
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['day_num'] = calendar['d'].map(transform_day_to_num)
map_day_date = calendar[['date','day_num']].set_index('day_num')['date']
sales_train_long_format['date'] = sales_train_long_format['day_num'].map(map_day_date)
list1 = ['wm_yr_wk','event_name_1', 'event_type_1', 'event_name_2', 'event_type_2','snap_CA', 'snap_TX', 'snap_WI','day_num']
sales_train_long_format = sales_train_long_format.merge(calendar[list1],on='day_num',how='left')

In [7]:
sales_train_long_format = sales_train_long_format.merge(sell_prices,how='left',on = ['store_id','item_id','wm_yr_wk'])
###把没卖的变成nan
sales_train_long_format.loc[pd.isna(sales_train_long_format.sell_price),'sale'] = np.nan
sales_train_long_format = reduce_mem_usage(sales_train_long_format)

Mem. usage decreased to 6526.92 Mb (25.0% reduction)


### part3 or  part4

- mean encoding part

In [8]:
sales_train = pd.read_csv('../data/sales_train_validation.csv')
for _ in [f'd_{i}' for i in range(1914,1970)]:
    sales_train[_] = np.nan

In [9]:
def group_by_key_on_value(df,key,windows,agg_func = [np.mean,np.std,np.median,np.max,np.min],feature_name = 'No name'):
    save_dict = {}
    d_start_columns = list(df.columns[df.columns.str.startswith('d_')])
    for key_value in tqdm(df[key].unique()):
        get_key_df = df[df[key] == key_value].copy()
        rolling_df = \
            get_key_df[d_start_columns].sum(axis=0,skipna=True).shift(1).rolling(windows,int(windows/2)).\
            agg(agg_func)
        #rolling_df = rolling_df.loc[f'd_{day_min}':f'd_{day_max}']
        rolling_df.index = [key_value+'_'+_ for _ in rolling_df.index]
        rolling_df_dict = rolling_df.to_dict(orient='index')
        save_dict.update(rolling_df_dict)
    dict_mean,dict_std,dict_median,dict_max,dict_min = {},{},{},{},{}
    for key,value in save_dict.items():
        dict_mean[key] = value['mean']
        dict_std[key] = value['std']
        dict_median[key] = value['median']
        dict_max[key] = values['amax']
        dict_min[key] = values['amin']
    return dict_mean,dict_std,dict_median,dict_max,dict_min

In [10]:
def group_by_two_key_on_value(df,key1,key2,windows,agg_func,feature_name):
    df['key'] = df[key1].astype(str) + '_' + df[key2].astype(str)
    key = 'key'
    save_dict = {}
    d_start_columns = list(df.columns[df.columns.str.startswith('d_')])
    for key_value in tqdm(df[key].unique()):
        get_key_df = df[df[key] == key_value].copy()
        rolling_df = \
            get_key_df[d_start_columns].sum(axis=0,skipna=True).shift(1).rolling(windows,int(windows/2)).\
            agg(agg_func)
        #rolling_df = rolling_df.loc[f'd_{day_min}':f'd_{day_max}']
        rolling_df.index = [key_value+'_'+_ for _ in rolling_df.index]
        rolling_df_dict = rolling_df.to_dict(orient='index')
        save_dict.update(rolling_df_dict)
    dict_mean,dict_std,dict_median = {},{},{}
    for key_temp,value in save_dict.items():
        dict_mean[key_temp] = value['mean']
        dict_std[key_temp] = value['std']
        dict_median[key_temp] = value['median']
    return dict_mean,dict_std,dict_median       

In [ ]:
def feature_engineer(df,num_core,date1):
    """
    df format:sales_train_long_format
    """
    df = df.copy()
    
    #### group by item_id
    for key_single in ['state_id','store_id','cat_id','dept_id','item_id']:
        df[f'{key_single}_day_num'] = df[key_single].astype(str) + '_d_' + df.day_num.astype(str)
        for day_interval in [7,14,28]:
            dict_mean,dict_std,dict_median,dict_max,dict_min = \
            group_by_key_on_value(sales_train,key_single,day_interval,[np.mean,np.std,np.median,np.max,np.min],\
                                  f'{key_single}_rolling_{day_interval}_sale')
            df[f'{key_single}_rolling_{day_interval}_sale_mean'] = df[f'{key_single}_day_num'].map(dict_mean)
            df[f'{key_single}_rolling_{day_interval}_sale_std'] = df[f'{key_single}_day_num'].map(dict_std)
            df[f'{key_single}_rolling_{day_interval}_sale_median'] = df[f'{key_single}_day_num'].map(dict_median)
            df[f'{key_single}_rolling_{day_interval}_sale_max'] = df[f'{key_single}_day_num'].map(dict_max)
            df[f'{key_single}_rolling_{day_interval}_sale_min'] = df[f'{key_single}_day_num'].map(dict_min)

            print(f'{key_single}_rolling_{day_interval}_sale','over')

    df = reduce_mem_usage(df,False)

    return df

In [ ]:
print(1)
train1 = feature_engineer(sales_train_long_format,12,None)
train1.to_pickle('data_part4.pkl')

In [15]:
def feature_engineer(df,num_core,date1):
    """
    df format:sales_train_long_format
    """
    df = df.copy()
    
    for double_key1,double_key2 in [
        ('state_id','cat_id'),
        ('state_id','dept_id'),
        ('state_id','item_id'),
        ('store_id','cat_id'),
        ('store_id','dept_id')]:
        df[f'{double_key1}_{double_key2}_day_num'] = df[double_key1].astype(str)+'_'+df[double_key2].astype(str) + '_d_' + df.day_num.astype(str)
        for day_interval in [7,14,28]:
            dict_mean,dict_std,dict_median = \
            group_by_two_key_on_value(sales_train,double_key1,double_key2,day_interval,['mean','std','median'],\
                                  f'{double_key1}_{double_key2}_rolling_{day_interval}_sale')
            df[f'{double_key1}_{double_key2}_rolling_{day_interval}_sale_mean'] = df[f'{double_key1}_{double_key2}_day_num'].map(dict_mean)
            df[f'{double_key1}_{double_key2}_rolling_{day_interval}_sale_std'] = df[f'{double_key1}_{double_key2}_day_num'].map(dict_std)
            df[f'{double_key1}_{double_key2}_rolling_{day_interval}_sale_median'] = df[f'{double_key1}_{double_key2}_day_num'].map(dict_median)
            print(f'{double_key1}_{double_key2}_rolling_{day_interval}_sale','over')

    df = reduce_mem_usage(df,False)

    return df

In [16]:
print(1)
train1 = feature_engineer(sales_train_long_format,12,None)
train1.to_pickle('data_part5.pkl')

1



state_id_cat_id_rolling_7_sale over


KeyboardInterrupt: 